##### Copyright 2021 The TensorFlow Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Object Detection in TensorFlow Model Analysis
***An Example of Calculating Object Detection Metrics with Tensorflow Model Analysis***

[TensorFlow Model Analysis (TFMA)](https://www.tensorflow.org/tfx/guide/tfma) is a library for performing model evaluation across different slices of data. TFMA performs its computations in a scalable distributed manner using [Apache Beam](https://beam.apache.org/documentation/programming-guide/).

This example colab notebook illustrates how  TFMA can be used to evaluate your model performance on object detection tasks. The metrics in this example follow the standard in the [COCO object detection contest](https://cocodataset.org/#home). We will run tensorflow model analysis in a model agnostic mode, meaning that TFMA doesn't call a SavedModel to get predictions, but looks materialized predictions in the incoming tf.Examples. The labels and predictions are included in our randomly generated examples.

You can use the tensorflow model analysis module to measure your model performance with different classes, maximum number of detections per image, and IoU(intersection over union) thresholds. In addition, you can take adavantage of the generic TFMA features to calculate metrics on the specific slices of the data, and do model comparison with other models. For more details, refer to [tfma basics colab](https://www.tensorflow.org/tfx/tutorials/model_analysis/tfma_basic).

Note: In order to understand TFMA and how it works with Apache Beam, you'll need to know a little bit about Apache Beam itself.  The <a target='_blank' href='https://beam.apache.org/documentation/programming-guide/'>Beam Programming Guide</a> is a great place to start.

## Install Jupyter Extensions
Note: If running in a local Jupyter notebook, then these Jupyter extensions must be installed in the environment before running Jupyter.

```bash
jupyter nbextension enable --py widgetsnbextension --sys-prefix 
jupyter nbextension install --py --symlink tensorflow_model_analysis --sys-prefix 
jupyter nbextension enable --py tensorflow_model_analysis --sys-prefix 
```

## Install TensorFlow Model Analysis (TFMA)

This will pull in all the dependencies, and will take a minute.


In [1]:
# Upgrade pip to the latest, and install TFMA.
!pip install -U pip
!pip install tensorflow-model-analysis

/bin/sh: line 1: pip: command not found
/bin/sh: line 1: pip: command not found


**Now you must restart the runtime before running the cells below.**

In [ ]:
# This setup was tested with TF 2.5 and TFMA 0.31 (using colab), but it should
# also work with the latest release.
import sys

# Confirm that we're using Python 3
assert sys.version_info.major==3, 'This notebook must be run using Python 3.'

import tensorflow as tf
print('TF version: {}'.format(tf.__version__))
import apache_beam as beam
print('Beam version: {}'.format(beam.__version__))
import tensorflow_model_analysis as tfma
print('TFMA version: {}'.format(tfma.__version__))

In [ ]:
import numpy as np
import os
import tempfile
from google.protobuf import text_format

Create a temp working directory.

In [ ]:
BASE_DIR = tempfile.gettempdir()

**NOTE: The output above should be clear of errors before proceeding. Re-run the install if you are still seeing errors. Also, make sure to restart the runtime/kernel before moving to the next step.**

## Create object detection dataset.
In this section, we fake the labels and predictions from open source COCO dataset.

In object detections, the object are presented with bounding boxes (bbox). The labels should provide the information of bounding boxes in the following order:

[left boundary, top boundary, right boundary, top boundary, class id].

The predictions should provide information with the confidence score of the predictions:

[left boundary, top boundary, right boundary, top boundary, class id, confidence score].

TFMA also supports concatenating these data if they are provided under separate keys. You need to pass the key names (in correct order) into the `labels_to_stack` and `predictions_to_stack` in the metric configs under [`metric_specs`](https://www.tensorflow.org/tfx/model_analysis/api_docs/python/tfma/MetricsSpec) for TFMA to correctly concatenate the data under these keys.

## Create TFRecords

We create a dataset with two images. We write the data into a TFRecords file. 

In [ ]:
# create two images with labels and predictions for object detections
extracts = [{
    'features': {
        'image/labels/bbox/xmin': [272.1, 181.23, 174.74],
        'image/labels/bbox/ymin': [200.23,  86.28,   0.  ],
        'image/labels/bbox/xmax': [424.07, 208.67, 435.78],
        'image/labels/bbox/ymax': [480.  , 159.81, 220.79],
        'image/labels/bbox/class_id': [  2.  ,   2.  ,   2.  ],
        'image/predictions/bbox/xmin': [271.20, 178.53, 167.96],
        'image/predictions/bbox/ymin': [178.86, 92.57, 9.97],
        'image/predictions/bbox/xmax': [429.5, 206.39, 442.79],
        'image/predictions/bbox/ymax': [459.57, 159.71, 235.07],
        'image/predictions/bbox/class_id': [  2.  ,   2.  ,   2.  ],
        'image/predictions/bbox/score': [0.64, 0.38, 0.95],
    }
}, {
    'features': {
        'image/labels/bbox/xmin': [473.07, 204.01, 0.43, 204.42],
        'image/labels/bbox/ymin': [395.93, 235.08, 499.79, 304.10],
        'image/labels/bbox/xmax': [503.07, 264.85, 340.22, 256.93],
        'image/labels/bbox/ymax': [424.60, 412.44, 606.24, 456.86],
        'image/labels/bbox/class_id': [1.  ,   2.  ,   2.  , 2.],
        'image/predictions/bbox/xmin': [471.15, 198.53, -32.86, 201.59],
        'image/predictions/bbox/ymin': [398.57, 242.14, 505.75, 299.39],
        'image/predictions/bbox/xmax': [502.29, 263.93, 338.82, 258.4],
        'image/predictions/bbox/ymax': [428.26, 427.51, 619.66, 452.88],
        'image/predictions/bbox/class_id': [1.  ,   2.  ,   2.  , 1.],
        'image/predictions/bbox/score': [0.54, 0.95, 0.17, 0.05],
    }
}]

# transform the images into tf examples
examples = []
for extract in extracts:
  features = {}
  for key, value in extract['features'].items():
    features[key] = tf.train.Feature(float_list=tf.train.FloatList(value=value))
  example = tf.train.Example(features=tf.train.Features(feature=features)).SerializeToString()
  examples.append(example)

# write examples into a tfrecord file     
tfrecord_file = os.path.join(BASE_DIR, 'object_detection_data.rio')
with tf.io.TFRecordWriter(tfrecord_file) as writer:
  for example in examples:
    writer.write(example)

!ls {tfrecord_file}

/tmp/object_detection_data.rio


## Setup and Run TFMA

TFMA supports a number of different [COCO object detection metrics](https://cocodataset.org/#detection-eval), including:
1. COCOMeanAveragePrecision
2. COCOMeanAverageRecall
3. COCOAveragePrecision
4. COCOAverageRecall

TFMA also supports several other confusion matrix metrics for object detections, including:
1. ObjectDetectionPrecisionAtRecall
2. ObjectDetectionPrecision
3. ObjectDetectionRecall
4. ObjectDetectionThresholdAtRecall

After creating a [`tfma.EvalConfig`](https://www.tensorflow.org/tfx/model_analysis/api_docs/python/tfma/EvalConfig) and [`tfma.EvalSharedModel`](https://www.tensorflow.org/tfx/model_analysis/api_docs/python/tfma/EvalSharedModel) (in this example, we are in the model agnostic mode) we can then run TFMA using [`tfma.run_model_analysis`](https://www.tensorflow.org/tfx/model_analysis/api_docs/python/tfma/run_model_analysis). This will create a [`tfma.EvalResult`](https://www.tensorflow.org/tfx/model_analysis/api_docs/python/tfma/EvalResult) which we can use later for rendering our metrics and plots.

In [ ]:
# Setup tfma.EvalConfig settings
eval_config = text_format.Parse("""
  ## Model information
  model_specs {
    signature_name: "serving_default"
    prediction_key: "predictions" # placeholder
    label_key: "labels" # placeholder
  }
  slicing_specs {
  }
  ## Metrics information
  metrics_specs {
    metrics {
      class_name: "COCOMeanAveragePrecision"
      config:'"iou_thresholds":[0.5], "class_ids":[1,2], "num_thresholds":3,'
            '"max_num_detections":100, "name":"MAP_iou0.5", '
            '"labels_to_stack":["image/labels/bbox/xmin", '
            '"image/labels/bbox/ymin", "image/labels/bbox/xmax", '
            '"image/labels/bbox/ymax", "image/labels/bbox/class_id"], '
            '"predictions_to_stack":["image/predictions/bbox/xmin", '
            '"image/predictions/bbox/ymin", "image/predictions/bbox/xmax", '
            '"image/predictions/bbox/ymax", "image/predictions/bbox/class_id", '
            '"image/predictions/bbox/score"]'
    }
    metrics {
      class_name: "ObjectDetectionPrecision"
      config:'"iou_threshold":0.5, "class_id":1, '
            '"max_num_detections":100, "name":"Precision_iou0.5", '
            '"thresholds":[0.5], '
            '"labels_to_stack":["image/labels/bbox/xmin", '
            '"image/labels/bbox/ymin", "image/labels/bbox/xmax", '
            '"image/labels/bbox/ymax", "image/labels/bbox/class_id"], '
            '"predictions_to_stack":["image/predictions/bbox/xmin", '
            '"image/predictions/bbox/ymin", "image/predictions/bbox/xmax", '
            '"image/predictions/bbox/ymax", "image/predictions/bbox/class_id", '
            '"image/predictions/bbox/score"]'
    }
  }
""", tfma.EvalConfig())

# This example is running in a model agnostic mode
# Create a temp output path
output_path = os.path.join(BASE_DIR, 'output_path')

# Run TFMA
object_detection_eval_result = tfma.run_model_analysis(
    eval_shared_model=None,
    eval_config=eval_config,
    data_location=tfrecord_file,
    output_path=output_path)

/usr/local/lib/python3.7/dist-packages/tensorflow_model_analysis/metrics/confusion_matrix_metrics.py:809: RuntimeWarning: invalid value encountered in true_divide
  precisions = tp / (tp + fp)
Instructions for updating:
Use eager execution and: 
`tf.data.TFRecordDataset(path)`


## Visualizing Metrics and Plots

Now that we've run the evaluation, let's take a look at our visualizations using TFMA.

In [ ]:
eval_result = object_detection_eval_result

### Rendering Metrics

To view metrics you use [`tfma.view.render_slicing_metrics`](https://www.tensorflow.org/tfx/model_analysis/api_docs/python/tfma/view/render_slicing_metrics)

By default the views will display the `Overall` slice. To view a particular slice you can either use the name of the column (by setting `slicing_column`) or provide a [`tfma.SlicingSpec`](https://www.tensorflow.org/tfx/model_analysis/api_docs/python/tfma/SlicingSpec).

The metrics visualization supports the following interactions:

* Click and drag to pan
* Scroll to zoom
* Right click to reset the view
* Hover over the desired data point to see more details.
* Select from four different types of views using the selections at the bottom.

For example, we'll be setting `slicing_column` to look at the `trip_start_hour` feature from our previous `slicing_specs`.

In [ ]:
tfma.view.render_slicing_metrics(eval_result)